In [21]:
import pandas as pd
import random
from caboose_nbr.upcfr import UPCFr
from caboose_nbr.evaluate_recommendation import evaluate
from caboose_nbr.evaluate_recommendation import evaluate

In [2]:
train_baskets = pd.read_csv('data/instacart_30k/train_baskets.csv.gz')
test_baskets = pd.read_csv('data/instacart_30k/test_baskets.csv')
valid_baskets = pd.read_csv('data/instacart_30k/valid_baskets.csv')

In [3]:
all_users = train_baskets['user_id'].tolist()
sample_users = random.sample(all_users,1000)

In [4]:
train_baskets = train_baskets[train_baskets['user_id'].isin(sample_users)]
test_baskets = test_baskets[test_baskets['user_id'].isin(sample_users)]
valid_baskets = valid_baskets[valid_baskets['user_id'].isin(sample_users)]

In [5]:
train_baskets.shape

(398187, 10)

In [6]:
train_baskets[['user_id','item_id']].drop_duplicates().shape

(123056, 2)

In [7]:
upcfr_similaripy = UPCFr(train_baskets, test_baskets, valid_baskets, mode = 'similaripy')
upcfr_similaripy.train()
upcfr_similaripy_preds = upcfr_similaripy.predict()

number of test users: 962
filtered items: 8898
/Users/mozhdeh/Documents/caboose/caboose_nbr/upcfr.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['uid'] = self.data['user_id'].rank(method='dense')-1
/Users/mozhdeh/Documents/caboose/caboose_nbr/upcfr.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['pid'] = self.data['item_id'].rank(method='dense')-1


uid       962
pid      8898
Score    1699
dtype: int64
(108170, 3)
962 8898


Done: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 962/962 [00:00<00:00, 18572.19it/s]

usersim shape (962, 962)



Done: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 962/962 [00:00<00:00, 5829.49it/s]


In [8]:
upcfr_caboose = UPCFr(train_baskets, test_baskets, valid_baskets, mode = 'caboose')
upcfr_caboose.train()
upcfr_caboose_preds = upcfr_caboose.predict()

number of test users: 962
filtered items: 8898
/Users/mozhdeh/Documents/caboose/caboose_nbr/upcfr.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['uid'] = self.data['user_id'].rank(method='dense')-1
/Users/mozhdeh/Documents/caboose/caboose_nbr/upcfr.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['pid'] = self.data['item_id'].rank(method='dense')-1


uid       962
pid      8898
Score    1699
dtype: int64
(108170, 3)
962 8898


--Creating transpose of R...
--Computing row norms...
--Configuring for top-k -- num_threads: 4; pinning? false;
--Scheduling parallel top-k computation...


In [17]:
def sanity_check(default_preds, caboose_preds):
    bad_preds  = []
    for key in default_preds:
        if default_preds[key] != caboose_preds[key]:
            bad_preds.append(key)
    print('bad predictions:',len(bad_preds))

In [18]:
sanity_check(upcfr_similaripy_preds,upcfr_caboose_preds)

bad predictions: 21


In [22]:
user_test_baskets_df = test_baskets.groupby('user_id')['item_id'].apply(list).reset_index()
user_test_baskets_dict = dict(zip( user_test_baskets_df['user_id'],user_test_baskets_df['item_id']))


In [23]:
_,_ = evaluate(user_test_baskets_dict,upcfr_similaripy_preds)

10
recall: 0.10274269521555535
ndcg: 0.1648943722123208
20
recall: 0.1490264199592029
ndcg: 0.12998109763569374


In [24]:
_,_ = evaluate(user_test_baskets_dict,upcfr_caboose_preds)

10
recall: 0.10274269521555535
ndcg: 0.1648943722123208
20
recall: 0.1490264199592029
ndcg: 0.12998109763569374
